In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import pandas as pd
import scipy
%config InlineBackend.figure_format = 'svg' 

from clickhouse_driver import Client

client = Client('oleg.orbita.work', port = 9000, user='student_ml',
password='CGRV8zemLGgsfdz7Uc6ZJeisGTcPQa', settings={'use_numpy': True})

table_name = 'students_datas. house_train'
df = client.query_dataframe(f"SELECT * FROM {table_name}")

pd.options.display.max_columns = None
pd.options.display.max_rows = None

2.1. Сколько данных в датасете?

2.2. Сколько параметров? Выведите список всех параметров.

In [ ]:
df.info(verbose=True, show_counts=False)  # 1460 записей, 81 полей

2.3. Есть ли категориальные признаки? Перечислите / выведите их.

In [ ]:
df.select_dtypes(include=['category']).info(
    show_counts=False)  # 27 категориальных полей

2.4. Выведите первые пять строчек DataFrame.

In [ ]:
df.head()

3 Проверьте, есть ли пропуски и повторы в данных.

3.1. Удалите повторы

3.2. Удалите столбцы в которых пропущено более 15% данных

In [ ]:
df = df.drop_duplicates()
df = df.dropna(axis=1, thresh=len(df) * 0.85)
df = df.dropna()
df.info(verbose=False, show_counts=True) # вместо 81 стало 74 поля

4 Постройте гистограмму параметра SalePrice. Подчиняется ли распределение нормальному?

In [ ]:
# гистограмма схожа, но не подчиняется нормальному распределению
df['SalePrice'].plot.hist()

In [ ]:
stat, p = scipy.stats.normaltest(df['SalePrice'])  # Критерий согласия Пирсона
print('Statistics=%.3f, p-value=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('Принять гипотезу о нормальном распределении')
else:
    print('Отклонить гипотезу о нормальном распределении')

In [ ]:
stat, p = scipy.stats.shapiro(df['SalePrice'])  # тест Шапиро-Уилк
print('Statistics=%.3f, p-value=%.3f' % (stat, p))
alpha = 0.05
if p > alpha:
    print('Принять гипотезу о нормальности')
else:
    print('Отклонить гипотезу о нормальности')

5 Построить коробочную диаграмму (ящик с усами) признака SalePrice всех домов в данных.

In [ ]:
# отображает то же, что и гистограмма - распределение не нормальное, есть выбросы
df.boxplot(column=['SalePrice'])

6 Постройте Гистограммы и Боксплоты по группам:

6.1. кондиционером ('CentralAir') и без кондиционера

In [ ]:
df.boxplot(column='SalePrice', by='CentralAir')

In [ ]:
# чуть более 'чистые' ящики с усами
Q1 = df['SalePrice'].quantile(0.25)
Q3 = df['SalePrice'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df_cl = df[(df['SalePrice'] >= lower_bound) & (df['SalePrice'] <= upper_bound)]
df_cl.boxplot(column='SalePrice', by='CentralAir')

6.2.цены продажи домов (параметр 'SalePrice'), сгруппированные по размеру гаража (параметр 'GarageCars')

In [ ]:
# df.boxplot(column='SalePrice', by='GarageCars')
df_cl.boxplot(column='SalePrice', by='GarageCars')

7 Постройте гистограмму частот:

7.1. частот размеров гаража

In [ ]:
sns.histplot(df['GarageCars'], bins=4, color="skyblue", edgecolor="black", alpha=0.7, kde=True)
plt.ylabel("Frequency", fontsize=12)
plt.show()

7.2. частот центрального кондиционирования

In [ ]:
sns.histplot(df['CentralAir'], color="skyblue", edgecolor="black")
plt.ylabel("Frequency", fontsize=12)
plt.show()

8 Рассчитайте долю домов

8.1. с продажной ценой между 25-м процентилем и 75-м процентилем

In [ ]:
Q1 = df['SalePrice'].quantile(0.25)
Q3 = df['SalePrice'].quantile(0.75)
IQR = Q3 - Q1
print((len(df[(df['SalePrice'] >= Q1) & (df['SalePrice'] <= Q3)]) / len(df['SalePrice'])) * 100)

8.2. с общей площадью в квадратных футах от 25-го процентиля до 75-го процентиля

In [ ]:
Q1 = df['LotArea'].quantile(0.25)
Q3 = df['LotArea'].quantile(0.75)
IQR = Q3 - Q1
print((len(df[(df['LotArea'] >= Q1) & (df['LotArea'] <= Q3)]) / len(df['LotArea'])) * 100)

9 Получите ковариационную матрицу для всех данных DataFrame и используйте анализ тепловой карты.

In [ ]:
df.cov(numeric_only=True)

In [ ]:
plt.rcParams["figure.figsize"] = (18, 15)
data = df.loc[:, ~df.columns.isin(['Id'])].corr(numeric_only=True)
sns.heatmap(data, mask=np.triu(np.ones_like(data)))

Выведетите 10 параметров с наибольшей корреляцией с SalePrice

In [ ]:
df.corr(numeric_only=True).abs().unstack().sort_values(kind="quicksort", ascending=False)["SalePrice"][:10]